<a href="https://colab.research.google.com/github/vitamingyu/ml-for-Healthcare-Analytics/blob/main/diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 분석순서
# 딥러닝, 그리드 탐색을 사용한 당뇨병 진단
# 데이터셋 소개
# 케라스 모델 설계
# 사이킷런으로 그리드 탐색 실행하기
# 드롭아웃 규제로 과적합줄이기
# 최적의 초매개변수 찾기
# 최적의 초매개변수를 사용해 예측하기

In [3]:
# 딥러닝 알고리즘을 사용해 당뇨병 발생을 예측하는 방법과 그리드 탐색법을 사용해 알고리즘을 최적화하는 방법을 다룬다
import sys
import pandas as pd
import numpy as np
import sklearn
from tensorflow import keras

In [4]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['n_pregnant', 'glucose_concentration', 'blood_pressure (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI', 'pedigree_function', 'age', 'class']

df = pd.read_csv(url, names=names)
df.describe()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
df[df['glucose_concentration']==0]
# 혈당이 0인 값(이상치) 필터링

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [8]:
columns=['glucose_concentration', 'blood_pressure (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']
for col in columns:
  df[col].replace(0, np.nan, inplace=True)  # 0을 결측값으로 대체
df.describe()  # count는 결측값이 아닌 행의 수

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [9]:
df.dropna(inplace=True)
df.describe()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [10]:
# 데이터 프레임을 Numpy 배열로 변환하기 위해 .values속성을 사용
dataset = df.values
print(dataset.shape)  # (392, 9)

(392, 9)


In [11]:
# Numpy배열은 인덱싱하기에 편리
X = dataset[:, 0:8]
Y = dataset[:, 8].astype(int)  # 정수로 변환

print(X.shape)  # (392, 8)
print(Y.shape)  # (392,)

print(X[:5])  # 부동소수점
print(Y[:5])  # 정수

(392, 8)
(392,)
[[1.000e+00 8.900e+01 6.600e+01 2.300e+01 9.400e+01 2.810e+01 1.670e-01
  2.100e+01]
 [0.000e+00 1.370e+02 4.000e+01 3.500e+01 1.680e+02 4.310e+01 2.288e+00
  3.300e+01]
 [3.000e+00 7.800e+01 5.000e+01 3.200e+01 8.800e+01 3.100e+01 2.480e-01
  2.600e+01]
 [2.000e+00 1.970e+02 7.000e+01 4.500e+01 5.430e+02 3.050e+01 1.580e-01
  5.300e+01]
 [1.000e+00 1.890e+02 6.000e+01 2.300e+01 8.460e+02 3.010e+01 3.980e-01
  5.900e+01]]
[0 1 1 1 1]


In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)  # 스케일링을 위한 행렬이 내부에서 만들어진다
# fit 메서드는 스케일링을 위한 파라미터를 계산하는 역할,  훈련 데이터를 이용해 평균과 표준편차를 계산, 이 정보를 이용하여 나중에 데이터를 표준화합니다.

# 훈련 데이터의 변환(표준화)
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()
# 정규화돼 평균 0, 표준편차 1을 가진다.
# 값들이 정규화돼 머신러닝 알고리즘이 모든 열에 대해 특별히 의존하거나 가중치를 부여하는 일 없이 균등한 값으로 처리할 수 있게 됨

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-9.063045e-18,1.132881e-17,-4.531523e-16,1.087565e-16,1.064908e-16,1.631348e-16,1.812609e-17,1.110223e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [25]:
!pip install scikeras

from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from keras.optimizers import Adam

In [26]:
# 모델 정의 과정을 사용자 정의 함수로 만들 것임
# 앞으로 매개변수를 바꿔가면서 모델을 여러 번 초기화해 재사용할 것이기 때문에 함수로 정의

def create_model():
  # 케라스 모델 정의
  model = Sequential()
  model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
  model.add(Dense(4, input_dim = 8 ,kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # 모델 컴파일
  adam = Adam(learning_rate = 0.01)
  model.compile(loss = 'bivary_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model

model = create_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 113 (452.00 Byte)
Trainable params: 113 (452.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# 위 신경망의 모든 매개변수의 개수는 113개이다 Total params: 113
